<a href="https://colab.research.google.com/github/merucode/RL/blob/93-Project-Trader-Custom_enviromnet/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do List

### 23.08.14

* [X] Make df and df_render
* [X] Observation Normalizatio(distinguish ohcl with volume)
* [X] Reward(profit) is extrected from df_render which is not normalization)
  * trade_world.py is changed
* [_] Add balance information
* [X] Add more holding action  
  * trade_world.py is changed

# Install Dependency and Test

In [1]:
!git clone -b 93-Project-Trader-Custom_enviromnet https://github.com/merucode/RL.git
!cd RL && mv * ../

Cloning into 'RL'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 258 (delta 78), reused 0 (delta 0), pack-reused 80
Receiving objects: 100% (258/258), 17.15 MiB | 6.21 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [16]:
!pip install gym==0.25.2
!pip install -e gym-examples

Obtaining file:///content/gym-examples
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gym-examples
    Found existing installation: gym-examples 0.0.1
    Uninstalling gym-examples-0.0.1:
      Successfully uninstalled gym-examples-0.0.1
  Running setup.py develop for gym-examples


### NOTE: After inistall gym-examples, Restart runtime

# STEP. Ready Data

In [19]:
import pandas as pd

df1 = pd.read_csv('KRW-BTC_5m_171001_200903_upbit.csv')
df2 = pd.read_csv('KRW-BTC_5m_200904_230810_upbit.csv')

df = pd.concat([df1, df2], ignore_index=True)
df = df[df.columns[1:]]
print(df.tail(3))

              open        high         low       close    volume
492909  39190000.0  39231000.0  39190000.0  39212000.0  2.929709
492910  39212000.0  39214000.0  39188000.0  39191000.0  3.407502
492911  39191000.0  39210000.0  39188000.0  39188000.0  3.884740


In [20]:
# Preporcessing df_render
df = df[:100000] #Temp : TEST
df_ohlc = df[df.columns[:4]] / 10000
df_v = df[df.columns[-1:]]
df = pd.concat([df_ohlc, df_v], axis=1)
df_render = df
print(df_render)

        open   high    low  close      volume
0      479.7  480.0  479.3  480.0    0.013444
1      479.8  479.9  479.2  479.3    0.013597
2      479.5  480.2  479.5  479.5    0.016304
3      480.1  480.2  479.5  480.1    0.017136
4      479.7  479.9  479.1  479.9    0.007615
...      ...    ...    ...    ...         ...
99995  377.0  377.1  374.5  376.1  132.139499
99996  376.9  377.8  376.1  377.1   41.992179
99997  377.1  378.6  376.6  378.1   28.228527
99998  378.1  378.5  377.1  378.3   14.609872
99999  378.3  378.3  377.4  378.3    9.547118

[100000 rows x 5 columns]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# Preporcessing df
df_ohlc = df[df.columns[:4]]
df_ohlc_normalization = (df_ohlc - df_ohlc.mean())/df_ohlc.std()
df_v = df[df.columns[-1:]]
df_v_normalization =  (df_v - df_v.mean())/df_v.std()
df = pd.concat([df_ohlc_normalization, df_v_normalization], axis=1)
print(df)

           open      high       low     close    volume
0     -1.130006 -1.128865 -1.131437 -1.129248 -0.615352
1     -1.129765 -1.129104 -1.131679 -1.130931 -0.615350
2     -1.130487 -1.128387 -1.130954 -1.130451 -0.615316
3     -1.129044 -1.128387 -1.130954 -1.129008 -0.615306
4     -1.130006 -1.129104 -1.131921 -1.129489 -0.615426
...         ...       ...       ...       ...       ...
99995 -1.376931 -1.375010 -1.384710 -1.379046  1.047038
99996 -1.377172 -1.373336 -1.380843 -1.376641 -0.087182
99997 -1.376691 -1.371422 -1.379635 -1.374237 -0.260354
99998 -1.374286 -1.371661 -1.378427 -1.373756 -0.431702
99999 -1.373806 -1.372140 -1.377702 -1.373756 -0.495401

[100000 rows x 5 columns]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Check env

In [5]:
import time
import matplotlib.pyplot as plt
import gym
import numpy as np

In [6]:
env = gym.make("gym_examples:TradeWorld-v0", df=df, obs_len=2016)
# 5m ohlcv obs_len=2016 is mean 7 days

# single env
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'observation shape : {env.observation_space.shape[0]}')
print(f'action shape      : {env.action_space.n}')

obs = env.reset()
action = 10
env.step(action)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


observation space : Box(0.0, inf, (10080,), float64)
action space      : Discrete(16)
observation shape : 10080
action shape      : 16


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float64, actual type: float32
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  

(array([-0.01231618, -0.01241297, -0.01223485, ..., -0.00929192,
        -0.00940871, -0.02168003], dtype=float32),
 0.03320494935282303,
 False,
 {'balance': 1000.0332049493528, 'time_step': 0, 'time_step_limit': 7968})

In [ ]:
envs = gym.vector.make("gym_examples:TradeWorld-v0", num_envs=2, df=df, obs_len=180)
obs_b = envs.reset()
actions = np.array([10, 5])
envs.step(actions)

/usr/local/lib/python3.10/dist-packages/gym/vector/vector_env.py:56: DeprecationWarning: WARN: Initializing vector env in old step API which returns one bool array instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


(array([[-1.61010718, -1.62276101, -1.59947574, ..., -1.53131783,
         -1.54626179, -0.32697502],
        [-1.61010718, -1.62276101, -1.59947574, ..., -1.53131783,
         -1.54626179, -0.32697502]]),
 array([0.02122165, 0.01673758]),
 array([False, False]),
 {'balance': array([1000.02122165, 1000.01673758]),
  '_balance': array([ True,  True])})

# STEP. Reinforce

In [22]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

#Device
device = device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(10080, 5040)
        self.fc2 = nn.Linear(5040, 2520)
        self.fc3 = nn.Linear(2520, 1260)
        self.fc4 = nn.Linear(1260, 16)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        #print(f"DEBUG 0: {x}")
        x = F.relu(self.fc1(x))
        #print(f"DEBUG 1: {x}")
        x = F.relu(self.fc2(x))
        #print(f"DEBUG 2: {x}")
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=0)
        #print(f"DEBUG 3: {x}")
        return x

    def put_data(self, item):
        self.data.append(item)

    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob + 1e-6) * R    # nan error handling : + 1e-6
            loss.backward()
        self.optimizer.step()
        self.data = []

In [23]:
import random

def action_eps(action, eps):
    coin = random.random()
    if coin < eps:
        action_e = random.randint(1,12)
    else:
        action_e = action
    return action_e

In [24]:
def main():
    env = gym.make("gym_examples:TradeWorld-v0", df=df, df_render=df_render, obs_len=2016)
    pi = Policy()
    score = 0.0
    max_score = -2000.0
    print_interval = 10
    eps = 0.5
    pi.to(device)

    for n_epi in range(3000):
        s = env.reset()
        done = False

        #print(f"DEBUG: episode:{n_epi}")
        while not done:
            #print(f"DEBUG: {s.shape}")
            prob = pi(torch.from_numpy(s).float().to(device))
            #print(f"DEBUG prob: {prob}")
            m = Categorical(prob)

            a = m.sample()
            a = action_eps(a.item(),eps)
            a = torch.tensor(a)

            s_prime, r, done, info = env.step(a.item())
            pi.put_data((r,prob[a]))
            s = s_prime
            score += r
            #print(f"DEBUG time step: {info['time_step']} / {info['time_step_limit']}")
            #print(f"DEBUG action: {a.item()}")
            #print(f"DEBUG reward: {r}")


        pi.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            if score/print_interval > max_score:
                torch.save(pi.state_dict(), 'model_weights.pth')
                print(f"# max avg balance : {1000 + score/print_interval}, and save model_weights.pth")
                max_score = score/print_interval
            score = 0.0
        if eps >= 0:
            eps = eps - 0.001

    env.close()

In [25]:
if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float64, actual type: float32
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  

# of episode :10, avg score : -1105.5699999999995
# max avg balance : -105.56999999999948, and save model_weights.pth
# of episode :20, avg score : -1016.3200000000004
# max avg balance : -16.32000000000039, and save model_weights.pth
# of episode :30, avg score : -1005.3199999999988
# max avg balance : -5.3199999999987995, and save model_weights.pth
# of episode :40, avg score : -1005.5899999999981
# of episode :50, avg score : -1005.2799999999972
# max avg balance : -5.279999999997244, and save model_weights.pth
# of episode :60, avg score : -1027.7500000000018
# of episode :70, avg score : -1006.0399999999978
# of episode :80, avg score : -1010.6899999999989
# of episode :90, avg score : -1005.370000000002
# of episode :100, avg score : -1015.0299999999974
# of episode :110, avg score : -1010.5299999999972
# of episode :120, avg score : -1034.5199999999993
# of episode :130, avg score : -1038.899999999999
# of episode :140, avg score : -1026.0899999999976
# of episode :150, avg scor

KeyboardInterrupt: ignored

# STEP. Render

In [26]:
def model_render(env_name, **kwargs):
    env = gym.make(env_name, **kwargs)
    episode_trigger=lambda e: True  # all Episode Recode

    env = gym.wrappers.RecordVideo(env, "./video", episode_trigger=episode_trigger, video_length=100000)

    for episode in range(1):
        s = env.reset()
        done = False
        score = 0

        pi = Policy()
        pi.load_state_dict(torch.load('model_weights.pth'))
        pi.eval()

        pi.to(device)

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            s_prime, r, done, info = env.step(a.item())
            done_mask = 0.0 if done else 1.0
            s = s_prime
            score += r

        print(f"# of episode {episode}:, avg score : {score}")
    env.close()

In [27]:
model_render("gym_examples:TradeWorld-v0", df=df, df_render=df_render, obs_len=2016)

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment TradeWorld-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float64, actual type: float32
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation s

# of episode 0:, avg score : -1001.8000000000006


In [28]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('./video/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=500 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Test Code